# Convert "Hey Nap" ONNX Model to TFLite

This notebook converts your trained `hey_nap.onnx` model to TFLite format for ESP32 deployment.

**Input:** `hey_nap.onnx` (your trained model)
**Output:** `hey_nap.tflite` (ready for ESP32)

## Step 1: Install Dependencies

In [ ]:
# Install conversion tools
!pip install -q onnx onnx2tf tensorflow
print("✅ Dependencies installed")

## Step 2: Upload Your ONNX Model

Click the "Choose Files" button below and select your `hey_nap.onnx` file.

In [ ]:
from google.colab import files
import os

print("📤 Please upload your hey_nap.onnx file...")
uploaded = files.upload()

# Get the uploaded file name
onnx_file = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {onnx_file}")
print(f"   File size: {os.path.getsize(onnx_file) / 1024:.1f} KB")

## Step 3: Convert ONNX to TFLite

In [ ]:
from pathlib import Path
import os

# Get model name from file
model_name = Path(onnx_file).stem
tflite_file = f"{model_name}.tflite"

print(f"🔄 Converting {onnx_file} to TFLite...")
print(f"   Output will be: {tflite_file}")
print("\nThis may take a minute...")

# Convert using onnx2tf
!onnx2tf -i {onnx_file} -o . -osd

# Check for output file (onnx2tf creates _float32.tflite)
output_file = f"{model_name}_float32.tflite"

if os.path.exists(output_file):
    # Rename to desired name
    os.rename(output_file, tflite_file)
    print(f"\n✅ Conversion successful!")
    print(f"   TFLite model: {tflite_file}")
    print(f"   File size: {os.path.getsize(tflite_file) / 1024:.1f} KB")
else:
    # Check if any TFLite file was created
    import glob
    tflite_files = glob.glob("*.tflite")
    if tflite_files:
        print(f"\n⚠️  Found TFLite file: {tflite_files[0]}")
        if tflite_files[0] != tflite_file:
            os.rename(tflite_files[0], tflite_file)
        print(f"✅ Renamed to: {tflite_file}")
    else:
        print("\n❌ TFLite file not found")
        print("\nChecking for errors...")
        print("Available files:")
        !ls -la

## Step 4: Download TFLite Model

In [ ]:
from google.colab import files
import os

tflite_file = f"{model_name}.tflite"

if os.path.exists(tflite_file):
    print(f"📥 Downloading {tflite_file}...")
    files.download(tflite_file)
    print(f"\n✅ Download complete!")
    print(f"\n🎉 Your TFLite model is ready!")
    print(f"\n📝 Next steps:")
    print(f"   1. Save the file to your project")
    print(f"   2. Test it: python3 test_hey_nap_local.py --model {tflite_file}")
    print(f"   3. Deploy to ESP32: cp {tflite_file} components/openwakeword/models/")
else:
    print(f"❌ TFLite file not found: {tflite_file}")
    print("\nChecking for any TFLite files...")
    import glob
    tflite_files = glob.glob("*.tflite")
    if tflite_files:
        print(f"Found: {tflite_files[0]}")
        files.download(tflite_files[0])
    else:
        print("No TFLite files found. Check the conversion step above for errors.")

## Troubleshooting

If conversion fails, try this alternative method:

In [ ]:
# Alternative conversion method (if onnx2tf fails)
print("Trying alternative conversion method...")
!pip install -q onnx onnx-tf

try:
    import onnx
    import tensorflow as tf
    from onnx_tf.backend import prepare
    import tempfile
    
    print("Loading ONNX model...")
    onnx_model = onnx.load(onnx_file)
    
    print("Converting to TensorFlow...")
    tf_rep = prepare(onnx_model)
    
    print("Exporting to SavedModel...")
    with tempfile.TemporaryDirectory() as tmp_dir:
        tf_model_path = os.path.join(tmp_dir, "saved_model")
        tf_rep.export_graph(tf_model_path)
        
        print("Converting to TFLite...")
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
        tflite_model = converter.convert()
        
        # Save
        tflite_file = f"{model_name}.tflite"
        with open(tflite_file, 'wb') as f:
            f.write(tflite_model)
        
        print(f"\n✅ Alternative conversion successful!")
        print(f"   File: {tflite_file}")
        print(f"   Size: {os.path.getsize(tflite_file) / 1024:.1f} KB")
        
        # Download
        from google.colab import files
        print(f"\n📥 Downloading {tflite_file}...")
        files.download(tflite_file)
        print(f"\n✅ Download complete!")
        
except Exception as e:
    print(f"\n❌ Alternative method failed: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 The ONNX model works fine for testing.")
    print("   You can use it with: python3 test_hey_nap_local.py --model hey_nap.onnx")